In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.feature
import skimage.exposure
import skimage.io
import skimage.transform
import sklearn.model_selection
import sklearn.metrics
import sklearn.svm

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
path_train = r'dataset/train'
path_valid = r'dataset/valid'

label_ref = {
    'black widow': 0,
    'captain america': 1,
    'doctor strange': 2,
    'hulk': 3,
    'ironman': 4,
    'loki': 5,
    'spider-man': 6,
    'thanos': 7
}

In [3]:
def get_img_cls(data_path):
    pic_labels = {}

    for hero in os.listdir(data_path):
        for pic in os.listdir(f'{data_path}/{hero}'):
            pic_labels[f'{data_path}/{hero}/{pic}'] = label_ref[hero]

    return pic_labels

In [4]:
hog_img_size = (128, 64)
hog_orientations = 8
hog_pixels_per_cell = (16, 16)
hog_cells_per_block = (1, 1)
hog_channel_axis = -1

def get_hog(resized_image):
    return skimage.feature.hog(resized_image,
                              orientations=hog_orientations,
                              pixels_per_cell=hog_pixels_per_cell,
                              cells_per_block=hog_cells_per_block,
                              channel_axis=hog_channel_axis)

In [5]:
train_img_cls = get_img_cls(path_train)

X_list = []
y_list = []

for img in train_img_cls.keys():
    img_data = skimage.io.imread(img)
    resized_img = skimage.transform.resize(img_data, hog_img_size)
    X_list.append(get_hog(resized_img))
    y_list.append(train_img_cls[img])

X = np.vstack(X_list)
y = np.array(y_list)

# X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=10)

In [6]:
print(X.shape, y.shape)

(2584, 256) (2584,)


In [7]:
valid_img_cls = get_img_cls(path_valid)

valid_X_list = []
valid_y_list = []

for img in valid_img_cls.keys():
    img_data = skimage.io.imread(img)
    resized_img = skimage.transform.resize(img_data, hog_img_size)
    valid_X_list.append(get_hog(resized_img))
    valid_y_list.append(valid_img_cls[img])

X_valid = np.vstack(valid_X_list)
y_valid = np.array(valid_y_list)

In [8]:
print(X_valid.shape, y_valid.shape)

(451, 256) (451,)


In [9]:
svm_C = 9
svm_gamma = 0.4 # rbf, poly, sigmoid
svm_kernel = 'rbf'
svm_degree = 3 # poly
svm_coef = 0 # poly, sigmoid
svm_decision = 'ovr'
svm_shrink = True

svm_model = sklearn.svm.SVC(C=svm_C,
                            gamma=svm_gamma, #
                            kernel=svm_kernel,
                            coef0=svm_coef,
                            decision_function_shape=svm_decision,
                            shrinking=svm_shrink,
                            verbose=True)
svm_model.fit(X, y)

y_pred = svm_model.predict(X_valid)
sklearn.metrics.accuracy_score(y_valid, y_pred)

[LibSVM]

0.2616407982261641

In [10]:
'''
hog_img_size = (128, 64)
hog_orientations = 8
hog_pixels_per_cell = (16, 16)
hog_cells_per_block = (1, 1)
hog_channel_axis = -1

svm_C = 9
svm_gamma = 0.4
svm_kernal = 'rbf'
svm_degree = 3
svm_coef = 0
svm_decision = 'ovr'

0.2616407982261641
'''

"\nhog_img_size = (128, 64)\nhog_orientations = 8\nhog_pixels_per_cell = (16, 16)\nhog_cells_per_block = (1, 1)\nhog_channel_axis = -1\n\nsvm_C = 9\nsvm_gamma = 0.4\nsvm_kernal = 'rbf'\nsvm_degree = 3\nsvm_coef = 0\nsvm_decision = 'ovr'\n\n0.2616407982261641\n"

In [11]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)